In [45]:
import os
import SimpleITK as sitk
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

import nibabel
from monai.transforms import Resize

import time

In [46]:
import sys
sys.path.insert(1, '/workspace')

In [55]:
from_dir = '/workspace/BoneMeta_all_192_VI/preds/[Best]bonemeta_fn_0_3D_Unet_DropRes_lv6_All192_model_epoch960/numpy'
to_dir = '/workspace/BoneMeta_all_192_VI/preds/[Best]bonemeta_fn_0_3D_Unet_DropRes_lv6_All192_model_epoch960/numpy_orig'
image_dir = '/workspace/BoneMeta_new/images'

os.makedirs(to_dir, exist_ok=True)

data_info_path = '/workspace/DataInfo/data_info_VI_2022_04_21.csv'
WHICH_SHAPE = 'Original Shape' # '1mm Resized Shape'

def get_label_path(file):
    return os.path.join(label_data_dir, file)

def get_image_path(file):
    return os.path.join(from_dir, file)

def case_to_file(case):
    return case+'.npy'

def file_to_case(file_name):
    return file_name.split('.')[0]

In [56]:
data_info = pd.read_csv(data_info_path, index_col=0)
data_info.tail()

,Case,Lesion Type,Lesion Location,Probability,I/V,File Name,Lesion Size,Background Size,Original Shape,1mm Resized Shape,Original Center,1mm Resized Center,Pixel Size,Spine Shape,Spine Resized Shape,Spine Center,Spine Resized Shape.1,X Spacing,Y Spacing,Slice Interval
Data,,,,,,,,,,,,,,,,,,,,
train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,C7,100,V,lesionAnnot3D-001.nii.gz,1708.3,53307.7,"[512, 512, 341]","[327, 327, 341]","[262, 261, 13]","[167, 166, 13]",4188,"[256, 256, 341]","[39, 84, 13]","[62, 133, 13]","[39, 84, 13]",0.6387,0.6387,-1.0
train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,T1,100,I,lesionAnnot3D-002.nii.gz,764.4,15435.6,"[512, 512, 341]","[327, 327, 341]","[322, 300, 7]","[205, 191, 7]",1874,"[256, 256, 341]","[77, 109, 7]","[122, 172, 7]","[77, 109, 7]",0.6387,0.6387,-1.0
train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,T7,100,V,lesionAnnot3D-004.nii.gz,2893.2,44356.8,"[512, 512, 341]","[327, 327, 341]","[258, 322, 155]","[164, 205, 155]",7093,"[256, 256, 341]","[37, 124, 155]","[58, 194, 155]","[37, 124, 155]",0.6387,0.6387,-1.0
train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,T9-1,100,V,lesionAnnot3D-005.nii.gz,3801.2,121246.8,"[512, 512, 341]","[327, 327, 341]","[262, 363, 218]","[167, 232, 218]",9319,"[256, 256, 341]","[39, 150, 218]","[62, 235, 218]","[39, 150, 218]",0.6387,0.6387,-1.0
train_add,SN055_20170301_CT_Chest_Low_Dose_+,L,Sternum,100,V,lesionAnnot3D-006.nii.gz,16.4,300411.6,"[512, 512, 341]","[327, 327, 341]","[298, 168, 87]","[190, 107, 87]",40313,"[256, 256, 341]","[62, 25, 87]","[98, 40, 87]","[62, 25, 87]",0.6387,0.6387,-1.0


In [57]:
SIZE_INFO_DF = data_info.loc[:,['Case', WHICH_SHAPE]]
SIZE_INFO_DF.drop_duplicates(inplace=True)
SIZE_INFO_DF.set_index('Case', inplace=True)
print(SIZE_INFO_DF.shape)

(341, 1)


In [58]:
SIZE_INFO_DF.head()

,Original Shape
Case,
000232_20190401_CT_Abdomen+Pelvis_Dynamic_(contrast),"[512, 512, 435]"
000232_20190423_Chest_CT_(contrast),"[512, 512, 315]"
000285_20190801_CT_Abdomen+Pelvis_Pre-Post_(contrast),"[512, 512, 528]"
000285_20190807_Chest_CT_(contrast),"[512, 512, 238]"
000316_20190627_Chest_CT_(contrast),"[512, 512, 283]"


In [59]:
def str_to_crz_list(resolution):
    crz = resolution.strip('[]').split(', ')
    return [float(value) for value in crz]

In [60]:
def save_image(np_image, target_dir, file_name):
    np_image = np_image.astype(np.int16)
    np.save(os.path.join(target_dir, file_name), np_image)
    
def save_label(np_image, target_dir, file_name):
    np_image = (np_image>0).astype(np.uint8)
    np.save(os.path.join(target_dir, file_name), np_image)

In [61]:
def str_to_size(size_str):
    crz = size_str.strip('[]').split(', ')
    torch_size = [int(value) for value in crz] 
    torch_size.reverse() # crz to zrc
    return torch_size

def get_size(file):
    case = file_to_case(file)
    size_str = SIZE_INFO_DF.loc[case][WHICH_SHAPE]
    if not type(size_str) == str:
        size_str = size_str[0]
    tensor_size = str_to_size(size_str)
    return tensor_size

def resample_all_1mm(np_image, target_size):
    resizer = Resize(target_size)
    resized_image = resizer(np_image[np.newaxis,:]) # with channel
    return resized_image[0] # without channel

def resample_all_1mm_label(np_label, target_size):
    np_label = np_label > 0.5
    resizer = Resize(target_size)
    resized_label = resizer(np_label[np.newaxis,:]) # with channel
    return resized_label[0] # without channel

In [66]:
from multiprocessing import Pool

def save_image(np_image, target_dir, file_name):
    np_image = np_image.astype(np.int16)
    np.save(os.path.join(target_dir, file_name), np_image)
    
def save_label(np_image, target_dir, file_name):
    np_image = (np_image>0.5).astype(np.uint8)
    np.save(os.path.join(target_dir, file_name), np_image)

def resample(file, file_dir, target_dir, resample_func, save_func):
    try:
        np_file = np.load(os.path.join(file_dir, file))
        target_size = get_size(file)
        resampled_image = resample_func(np_file, target_size)
        save_func(resampled_image, target_dir, file)
    except:
        return None
def wrapper(args):
    return resample(*args)

t = time.perf_counter()

file_list = os.listdir(from_dir)
arg_list = [(file, from_dir, to_dir, resample_all_1mm, save_image) for file in file_list]
pool = Pool(processes=32)
pool.map(wrapper, arg_list)
pool.close()
pool.join()

elapsed_time = time.perf_counter() - t

print(f'''
consumed_time: {elapsed_time}s
''')


consumed_time: 6.12782047491055s



In [ ]:
file_names = os.listdir(to_dir)
resampled_sizes = [np.load(os.path.join(to_dir, file)).shape for file in file_names]
image_sizes = [np.load(os.path.join(image_dir, file)).shape for file in file_names]

In [ ]:
image_sizes == resampled_sizes